In [1]:
from pyspark import  SparkContext
sc = SparkContext('local', 'App')

sc.master

'local'

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

ratings_df = pd.read_csv('/home/work/BigDataPractice/RecommendationEngine/ml-20m/ratings.csv', nrows =10000)
movies_df = pd.read_csv('/home/work/BigDataPractice/RecommendationEngine/ml-20m/movies.csv', nrows =10000)

movies_df['movieRow'] = movies_df.index #添加电影行数

In [3]:
ratings_df.tail()

,userId,movieId,rating,timestamp
9995,91,2797,3.5,1112061221
9996,91,2857,4.0,1111966776
9997,91,2858,4.5,1111557477
9998,91,2863,4.5,1111558557
9999,91,2890,2.5,1113202901


In [4]:
movies_df.tail()

,movieId,title,genres,movieRow
9995,32864,"Move Over, Darling (1963)",Comedy|Romance,9995
9996,32866,Love Me or Leave Me (1955),Drama|Musical|Romance,9996
9997,32875,Holiday (Jour de fête) (1949),Comedy,9997
9998,32882,"Big Store, The (1941)",Comedy|Musical,9998
9999,32890,Mother Küsters Goes to Heaven (Mutter Küsters'...,Drama,9999


# 筛选movies_df中的特征

In [5]:
movies_df = movies_df[['movieRow', 'movieId', 'title']]
movies_df.to_csv('moviesProcessed.csv', index = False, header = True, encoding = 'utf-8') 

# 将ratings_df中的moviesId 替换成行号

In [6]:
ratings_df = pd.merge(ratings_df, movies_df, on = 'movieId')

# 筛选ratings_df中的特征

In [7]:
ratings_df = ratings_df[['userId','movieRow','rating']]
ratings_df.to_csv('ratingsProcessed.csv',index = False,header = True,encoding = 'utf-8')

# 创建电影评分矩阵rating和评分记录矩阵record

In [8]:
userNo = ratings_df['userId'].max() + 1
movieNo = ratings_df['movieRow'].max() + 1
rating = np.zeros((movieNo, userNo))
flag = 0 #记录处理进度
ratings_df_length = np.shape(ratings_df)[0]
for index, row in ratings_df.iterrows():#获取ratings_df的每一行
        rating[int(row['movieRow']), int(row['userId'])] = row['rating']
        flag += 1 #表示处理完一行
#         print('processed %d, %d left'%(flag,ratings_df_length-flag))

In [9]:
#将构建record矩阵并将矩阵中的Boolean值改成相应的数值
record = rating > 0 
record = np.array(record, dtype = int)

# 构建模型

In [10]:
def normalizeRatings(rating, record): #缩放评分矩阵范围
    m,n = rating.shape #m为电影数量，n为用户数量
    rating_mean = np.zeros((m, 1)) #初始化电影评分平均值为0
    rating_norm = np.zeros((m, n)) #保存处理后的数据
    for i in range(m):
        idx = record[i, :] != 0 #获取每部电影评分用户的下标，每部电影的评分
        rating_mean[i] = np.mean(rating[i, idx]) #表示第i行已经评过分的用户的平均值
        rating_norm[i, idx] -= rating_mean[i]
    return rating_norm, rating_mean

In [11]:
rating_norm,rating_mean = normalizeRatings(rating, record) #由于数据中的某些行有这里会出现警告
#处理nan
rating_norm = np.nan_to_num(rating_norm)
rating_mean = np.nan_to_num(rating_mean)
#假设一共有10类电影
num_features = 10
#初始化电影内容矩阵和用户喜好矩阵，产生的参数都是随机数并且是正态分布的
X_parameters = tf.Variable(tf.random_normal([movieNo, num_features], stddev = 0.35))
Theta_parameters = tf.Variable(tf.random_normal([userNo, num_features], stddev = 0.35))
# 构建损失函数
loss = 1/2 * tf.reduce_sum(((tf.matmul(X_parameters, Theta_parameters, transpose_b = True)
                             - rating_norm)*record)**2) + 1/2 * (tf.reduce_sum(X_parameters**2) 
                                                               + tf.reduce_sum(Theta_parameters**2))
#将X_parameters，Theta_parameters矩阵相乘相乘之前将Theta_parameters转置
#创建优化器和优化目标
optimizer = tf.train.AdamOptimizer(1e-4)
train = optimizer.minimize(loss)

/root/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/root/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Instructions for updating:
Colocations handled automatically by placer.


# 训练模型

In [12]:
# 使用tensorboard查看loss经迭代后的改变
tf.summary.scalar('loss', loss)
summaryMerged = tf.summary.merge_all()
filename = 'movie_tensorboard'
writer = tf.summary.FileWriter(filename)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(5000):
    _, movie_summary = sess.run([train, summaryMerged])
    writer.add_summary(movie_summary, i)

# 评估模型

In [13]:
Current_X_parameters, Current_Theta_parameters = sess.run([X_parameters, Theta_parameters])
predicts = np.dot(Current_X_parameters, Current_Theta_parameters.T) + rating_mean
errors = np.sqrt(np.sum((predicts - rating) ** 2))

# 构建推荐系统

In [17]:
user_id = input('请输入要推荐的用户编号：')
sortedResult = predicts[:, int(user_id)].argsort()[::-1]#获取该用户的电影评分列表
idx =0
print('该用户推荐的评分最高的10部电影是：'.center(80,'='))
for i in sortedResult:
    print('评分：%.2f,电影名：%s'%(predicts[i,int(user_id)], movies_df.iloc[i]['title']))
    idx +=1
    if idx == 10:break

请输入要推荐的用户编号：66
===============================该用户推荐的评分最高的10部电影是：===============================
评分：5.93,电影名：Taking of Pelham One Two Three, The (1974)
评分：5.64,电影名：Cup, The (Phörpa) (1999)
评分：5.45,电影名：Coyote Ugly (2000)
评分：5.25,电影名：Iron Eagle (1986)
评分：5.11,电影名：Master and Commander: The Far Side of the World (2003)
评分：4.82,电影名：Great Dictator, The (1940)
评分：4.80,电影名：Bat*21 (1988)
评分：4.65,电影名：Transporter, The (2002)
评分：4.52,电影名：Man and a Woman, A (Un homme et une femme) (1966)
评分：4.43,电影名：Lilo & Stitch (2002)
